In [2]:
import pandas as pd
import numpy as np
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(
         'C:/Users/Cody/Downloads/usinflation-1dd0de4b5ee6.json', scope) # Your json file here

gc = gspread.authorize(credentials)

wks = gc.open("inflationRates 1914-2019").sheet1

rawData = wks.get_all_values()


In [4]:
df = pd.DataFrame(rawData)

# tmp = 1914
# years = np.zeros((df.shape[0]))
# for i in range (df.shape[0]):
#     years[i] = tmp
#     tmp = tmp + 1
tmp = 1914
years = [tmp]
for i in range (df.shape[0]-1):
    tmp = tmp + 1
    years.append(tmp)

for column in range(df.shape[1]):
    for row in range(df.shape[0]):
        if row != (df.shape[0] - 1):
            df[column][row] = re.findall('\d+\.\d+', df[column][row])[0]

for column in range(11):
    df[column][df.shape[0] - 1] = re.findall('\d+\.\d+', df[column][df.shape[0] - 1])[0]


df.index = years
columns = ["January", "February", "March", "April","May", "June", "July", "August", "September", "October",
              "November", "December"]
df.columns = columns
df

,January,February,March,April,May,June,July,August,September,October,November,December
1914,2.04,1.02,1.02,0.00,2.06,1.02,1.01,3.03,2.00,1.00,0.99,1.00
1915,1.00,1.01,0.00,2.04,2.02,2.02,1.00,0.98,0.98,0.99,0.98,1.98
1916,2.97,4.00,6.06,6.00,5.94,6.93,6.93,7.92,9.90,10.78,11.65,12.62
1917,12.50,15.38,14.29,18.87,19.63,20.37,18.52,19.27,19.82,19.47,17.39,18.10
1918,19.66,17.50,16.67,12.70,13.28,13.08,17.97,18.46,18.05,18.52,20.74,20.44
...,...,...,...,...,...,...,...,...,...,...,...,...
2015,0.09,0.03,0.07,0.20,0.04,0.12,0.17,0.20,0.04,0.17,0.50,0.73
2016,1.37,1.02,0.85,1.13,1.02,1.00,0.83,1.06,1.46,1.64,1.69,2.07
2017,2.50,2.74,2.38,2.20,1.87,1.63,1.73,1.94,2.23,2.04,2.20,2.11
2018,2.07,2.21,2.36,2.46,2.80,2.87,2.95,2.70,2.28,2.52,2.18,1.91


Cpi calculates the cpi from the initial date to December 2019

In [11]:
def cpi(initMonth, initYear, cpiInit):
    columns = ["January", "February", "March", "April","May", "June", "July", "August", "September", "October",
              "November", "December"]
    cpi = [cpiInit]
    month = initMonth
    for year in range(2020 - initYear):# 2019 -> initial date
        while month != 'December':
            cpi.append(cpiInit + (float(df[month][initYear])/100) * cpiInit / 12)
            cpiInit = (cpiInit + (float(df[month][initYear])/100) * cpiInit / 12)
            month = columns[columns.index(month) + 1]
        cpi.append(cpiInit + (float(df[month][initYear])/100) * cpiInit / 12)
        cpiInit = (cpiInit + (float(df[month][initYear])/100) * cpiInit / 12)
        month = 'January'
        year += 1
        initYear += 1
    return cpi[len(cpi) - 1]


a = cpi('January', 2001, 100)
print(a)
a = cpi('January', 1985, 100)
print(a)
a = cpi('January', 1977, 100)
print(a)
a = cpi('January', 1950, 100)
print(a)
a = cpi('January', 1930, 100)
print(a)
a = cpi('January', 1914, 100)
print(a)

151.28738103680922
252.85979855784132
473.36037973206646
1183.454921998616
3244.409635421597
9191.78375771597
